In [1]:
# 데이터 가져오기

raw_data = pd.read_csv('../../1. raw_data/KRI-DAC_Jeju_data5.txt', sep=',')
raw_data.tail()

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y
273178,273179,273179,202005,제주특별자치도,제주시,중소,침구수예점,x시,17000,0,1,0,949616.6171,1.502505e+06
273179,273180,273180,202005,제주특별자치도,제주시,중소1,서양음식,x시,12000,0,1,0,949665.1566,1.502805e+06
273180,273181,273181,202005,제주특별자치도,제주시,영세,편의점,x시,4800,0,1,0,950266.5976,1.502458e+06
273181,273182,273182,202005,제주특별자치도,제주시,중소,제과점,x시,25000,0,2,0,950524.6587,1.500760e+06
273182,273183,273183,202005,제주특별자치도,제주시,영세,서양음식,x시,6000,0,1,0,950674.5989,1.500760e+06


In [2]:
df = raw_data.copy()

In [3]:
from pyproj import Proj, transform

# ITRF좌표계
proj_ITRF = Proj(init='epsg:5179')
# WGS 좌표계 
proj_WGS84 = Proj(init='epsg:4326')


In [4]:
# 변환 예시 
x1, y1 = 961114.519726,1727112.269174
x2, y2 = transform(proj_ITRF, proj_WGS84, x1, y1)
x2, y2

(127.07103374451121, 35.53921546645965)

In [5]:
df['lon'], df['lat'] = transform(proj_ITRF, proj_WGS84, df['POINT_X'], df['POINT_Y'])
df.tail()

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lon,lat
273178,273179,273179,202005,제주특별자치도,제주시,중소,침구수예점,x시,17000,0,1,0,949616.6171,1.502505e+06,126.957495,33.513247
273179,273180,273180,202005,제주특별자치도,제주시,중소1,서양음식,x시,12000,0,1,0,949665.1566,1.502805e+06,126.958001,33.515956
273180,273181,273181,202005,제주특별자치도,제주시,영세,편의점,x시,4800,0,1,0,950266.5976,1.502458e+06,126.964496,33.512854
273181,273182,273182,202005,제주특별자치도,제주시,중소,제과점,x시,25000,0,2,0,950524.6587,1.500760e+06,126.967369,33.497551
273182,273183,273183,202005,제주특별자치도,제주시,영세,서양음식,x시,6000,0,1,0,950674.5989,1.500760e+06,126.968983,33.497564


In [6]:
df['lon'] = df['lon'].astype(str)
df['lat'] = df['lat'].astype(str)
df['lon_lat'] = df['lon']+'_'+df['lat']
df['lon_lat'].value_counts()

126.52427851055405_33.51290574138535     400
126.52935695016222_33.49899169591379     314
126.52880376641139_33.499886709171996    309
126.47793511858133_33.48574932701181     275
126.56362561835856_33.24739705179205     258
                                        ... 
126.82859256139503_33.333574147082494      1
126.33357199238228_33.241825154004715      1
126.5488735126737_33.49020581068737        1
126.5311000735163_33.491348968224116       1
126.95324437124411_33.508702714111564      1
Name: lon_lat, Length: 13820, dtype: int64

In [7]:
################# 좌표 정보 주소 변환 ####################
import requests
import json

In [8]:
url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.968983&y=33.497564"
headers = {"Authorization": "KakaoAK 1fecb2a27219321b92e1464ad08365fb"}

In [9]:
api_test = requests.get(url, headers=headers)
url_text = json.loads(api_test.text)
url_text

{'meta': {'total_count': 2},
 'documents': [{'region_type': 'B',
   'code': '5011033021',
   'address_name': '제주특별자치도 제주시 우도면 연평리',
   'region_1depth_name': '제주특별자치도',
   'region_2depth_name': '제주시',
   'region_3depth_name': '우도면',
   'region_4depth_name': '연평리',
   'x': 126.9588510304104,
   'y': 33.50152515199191},
  {'region_type': 'H',
   'code': '5011033000',
   'address_name': '제주특별자치도 제주시 우도면',
   'region_1depth_name': '제주특별자치도',
   'region_2depth_name': '제주시',
   'region_3depth_name': '우도면',
   'region_4depth_name': '',
   'x': 126.95338133967611,
   'y': 33.5065180993215}]}

In [10]:
dongmyeon = url_text['documents'][0]['region_3depth_name']
ri = url_text['documents'][0]['region_4depth_name']

dongmyeon, ri 

('우도면', '연평리')

In [11]:
df['lon'] = df['lon'].astype(str)
df['lat'] = df['lat'].astype(str)

In [12]:
df['url1'] = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=' + df['lon']
df['url2'] = '&y=' + df['lat']
df['url'] = df['url1'] + df['url2']

In [13]:
df.drop(columns=['url1','url2'],axis=1,inplace=True)

In [14]:
# dong_ls = []
# ri_ls = []
# for i in df.index[:100]:
#     url = df['url'][i]
#     headers = {"Authorization": "KakaoAK 1fecb2a27219321b92e1464ad08365fb"}
#     api_test = requests.get(url, headers=headers)
#     url_text = json.loads(api_test.text)
#     dong = url_text['documents'][0]['region_3depth_name']
#     dong_ls.append(dong)

In [15]:
##############################시각화#########################

In [16]:
df[df['lon_lat']=='126.17889362710162_33.30231529076243'].head()
# 같은 좌표인 경우에도 다른 업종인 경우가 많음 

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lon,lat,lon_lat,url
0,1,1,202005,제주특별자치도,제주시,영세,일반한식,00시,363000,66500,10,2,877005.9834,1479766.073,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,https://dapi.kakao.com/v2/local/geo/coord2regi...
4903,4904,4904,202005,제주특별자치도,제주시,영세,일반한식,01시,86500,0,3,0,877005.9834,1479766.073,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,https://dapi.kakao.com/v2/local/geo/coord2regi...
8568,8569,8569,202005,제주특별자치도,제주시,영세,일반한식,02시,208000,0,5,0,877005.9834,1479766.073,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,https://dapi.kakao.com/v2/local/geo/coord2regi...
11269,11270,11270,202005,제주특별자치도,제주시,영세,일반한식,03시,18000,0,1,0,877005.9834,1479766.073,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,https://dapi.kakao.com/v2/local/geo/coord2regi...
19121,19122,19122,202005,제주특별자치도,제주시,중소,한의원,07시,4800,0,2,0,877005.9834,1479766.073,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,https://dapi.kakao.com/v2/local/geo/coord2regi...


In [17]:
# 업종에 따른 folium 분포 나타내기 

In [18]:
df['FranClass'].value_counts()

영세     152294
일반      35889
중소      33211
중소1     32814
중소2     18975
Name: FranClass, dtype: int64

In [19]:
df_js2 = df[df['FranClass']=='중소2'] #중소2에 대해 추출 
df_js2 = df_js2[['SIGUNGU','FranClass','Type','lon','lat','lon_lat']] #업종별 분포 확인에 대한 정보만 추출 
df_js2.drop_duplicates(inplace=True) #중복되는 row 삭제 
df_js2 #중소2 업종 리스트 

,SIGUNGU,FranClass,Type,lon,lat,lon_lat
19,제주시,중소2,편의점,126.23936109660238,33.39198817926645,126.23936109660238_33.39198817926645
33,제주시,중소2,편의점,126.24355916544066,33.39700608601108,126.24355916544066_33.39700608601108
64,서귀포시,중소2,편의점,126.25136650558173,33.22131387143015,126.25136650558173_33.22131387143015
138,제주시,중소2,편의점,126.26146684798863,33.415288889899365,126.26146684798863_33.415288889899365
175,제주시,중소2,슈퍼마켓,126.26422533722761,33.411721332976384,126.26422533722761_33.411721332976384
...,...,...,...,...,...,...
211041,서귀포시,중소2,편의점,126.30552722476865,33.249560662589616,126.30552722476865_33.249560662589616
218169,제주시,중소2,홍삼제품,126.52427851055405,33.51290574138535,126.52427851055405_33.51290574138535
227687,서귀포시,중소2,기타숙박업,126.30555329730933,33.24820866355325,126.30555329730933_33.24820866355325
233602,제주시,중소2,상품권,126.52043562152693,33.517367797742295,126.52043562152693_33.517367797742295


In [20]:
#############folium을 통한 표현############ 
import pandas as pd 
import folium
from folium.plugins import MarkerCluster, MiniMap

In [21]:
map_jeju = folium.Map((33.38773221915759, 126.54124720118492), zoom_start= 10)
mc = MarkerCluster()

mc.add_child(folium.Marker(location=[33.48908804243811,126.49294055515932], popup='중소2', icon=folium.Icon(color='red')))

minimap = MiniMap()
map_jeju.add_child(minimap)

map_jeju.add_child(mc)